# 1. <a id='toc1_'></a>[Import packages](#toc0_)

In [ ]:
import xarray as xr

%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [ ]:
# Change to directory where data is stored
data_dir = "/kyukon/data/gent/courses/2025/computphys_C004504/input"
# For people following the old course: "/kyukon/data/gent/courses/2025/computfys_C001827/input"

# 2. <a id='toc2_'></a>[Open simulation data](#toc0_)

- Open the 2-meter temperature (variable `tas`) data for the `initSFX`-simulation starting at midnight on 1 July 2019 and running for 720 hours. To that end, use the function `open_data` below which expects a list of variable names (`var_name_list`) and a list of simulation names (`run_name_list`). This function will also automatically load in some other variables, which are used later in the practical.
- Which units are used for the temperature? Convert these units to degrees Celsius (if necessary) for this exercise. This will facilitate the interpretation of results.
- Let's now focus on one location: the Royal Meteorological Institute of Belgium (RMI) in Uccle. Select the temperature data for this location from the simulation dataset. Do so by finding the point in the simulation grid, which lies closest to the RMI. The coordinates of the RMI are 50.799 N, 4.359 E.
- From 22 until 26 July 2019, we experienced a short, but very intense heat wave in Belgium. The RMI speaks of a heat wave when the maximum temperatures in Uccle reach at least 25 degrees for at least 5 consecutive days, with at least 30 degrees  being reached on at least three days (see [heat wave definition](https://www.meteo.be/nl/info/weerwoorden/hittegolf) in Dutch). Select this 5-day period from the data and verify visually if these conditions for a heat wave are met in the simulation data by inspecting a plot.

<details><summary>Hints</summary> 

- The units of the variable `tas` can be found in the attributes.
- Selecting data in a point can be done with the `.sel()` function.
- Finding the grid point closest to the given coordinates means that `method="nearest"`.
- Selecting data over a given period can also be done with the `.sel()` function.
- To help your visual inspection, use the `ax.axhline(k)` function to plot a horizontal line at `y=k`.
</details>

In [ ]:
def open_data(data_dir, var_name_list, run_name_list):
    """
    Open multiple simulation datasets and merge them into a single xarray Dataset.

    Parameters
    ----------
    data_dir : str
        Directory where the data files are stored.
    var_name_list : list of str
        List of variable names to be opened from each simulation.
    run_name_list : list of str
        List of simulation run names to be opened.
    """

    ds_list_list = []
    ds_land_use_list = []

    for run_name in run_name_list:
        ds_list = []

        for var_name in var_name_list:
            ds = xr.open_dataset(
                f"{data_dir}/{run_name}_2019070100_720_{var_name}_2019-07-01T01_2019-07-31T00_3600_regridded.nc",
                chunks="auto",
            )
            ds_list.append(ds)

        ds = xr.merge(ds_list, compat="minimal")
        ds_list_list.append(ds)

        # Open the corresponding land use data (see next exercise)
        ds_land_use = xr.open_dataset(
            f"{data_dir}/{run_name}_landuse_regridded.nc",
            chunks="auto",
        )
        ds_land_use_list.append(ds_land_use)

    ds = xr.concat(ds_list_list, dim="run_name")
    ds_land_use = xr.concat(ds_land_use_list, dim="run_name")

    return xr.merge([ds, ds_land_use])

In [ ]:
def convert_to_celsius(ds, a):
    """
    Convert temperature variables in the dataset to degrees Celsius.

    Parameters
    ----------
    ds : xarray.Dataset
        Dataset containing temperature variables to be converted.
    a : float
        Value to subtract from temperature variables to convert to Celsius.
    """

    for var_name in ["tas", "ts", "tsl1", "tsl2", "troad1"]:
        if var_name in ds:
            old_attrs = ds[var_name].attrs
            ds[var_name] = ds[var_name] - a
            ds[var_name].attrs = old_attrs
            ds[var_name].attrs["units"] = "°C"

    return ds

# 3. <a id='toc3_'></a>[Investigate UHI](#toc0_)

In this next part of the practical, we will investigate the Urban Heat Island (UHI) in the simulations. We will do so by comparing the simulation to the MOCCA observational network. This network consists of 6 stations located in the city of Ghent. Each station is placed in a different urban environment, to gauge the influence of this environment on the micro-climate. One of these stations is located outside of the city, in Melle, and can be considered a rural station. We can define the UHI intensity by calculating the temperature difference between an urban and a rural station. This difference in temperature depends on the time of day. Therefore, the UHI is often studied by computing a so-called diurnal (or daily) cycle. This diurnal cycle is constructed by taking an average of the data over a certain period for each hour of the day. For example, all the data at midnight are averaged together, the data at 1:00, ... This diurnal cycle represents the variations throughout the day.

For this exercise, we will work with the `tas`-variable from the `initSFX`-simulation from 1 July 2019 (720 hours). We will also need the land use information stored in `initSFX_landuse_regridded.nc`. This land use data is automatically loaded by the `open_data`-function.

- Load in the MOCCA station data for the period of the heat wave. These data are stored in the file `MOCCA_tas_2019-07-22T01_2019-07-27T00.nc`. Explore the data.
- Load in the simulation data together with the land use information with the function called `open_data`, which takes the data directory, variable names and run names and loads in the corresponding variables and land use information as one dataset.
- Take a look at the variables from the land use data. How is this land use stored in the dataset?
- We will now consider the locations from the observational stations in the simulation data. Select the points in the simulation data which are closest to each of the stations with a for-loop. Assign coordinates so that you get an xarray DataSet with the same dimensions as the observations: `(time, location)`
- For each of the locations, consider the town fraction. Select the locations with the lowest and highest town fraction from the stations. We will use these to represent rural and urban locations respectively.
- For the period of the heat wave (22-26 July 2019), calculate the diurnal cycle of temperature for both locations for the simulation data. Plot the difference between the urban and rural diurnal cycles. Is a urban heat island present in the simulations? What is the UHI intensity? At what hours of the day is it the highest?
- Repeat the previous step for the observational data. Compare this result with the simulation data.
- Where does the difference in UHI intensity between the simulation and observations come from? Plot the bias of the diurnal cycle in temperature for both locations. Link the difference in UHI between the simulation and observations to the bias at the urban and rural locations.

<details><summary>Hints</summary> 

- Opening a dataset can be done with the `open_dataset()` function from xarray.
- The land use variables are those starting with `frac_`.
- The for-loop should work as follows:
    - Before the for-loop, make an empty list `ds_locations_list` to save the point data in.
    - Loop over each location in the MOCCA data `obs`, found in `obs.location`
    - Select the corresponding `lat` and `lon` values from the MOCCA data.
    - Use these values to select the nearest grid point from the simulation data with `.sel()`.
    - Assign a new coordinate called `location` to this selected data with `.assign_coords(location=...)`
    - Save this point data to the list.
    - Convert the list to an xarray DataSet called `ds_locations` with `xr.concat(ds_locations_list, dim="location")`
- The town fraction is store in `frac_town` in `ds_locations`. Print the 6 values and inspect which are smallest and largest.
- To calculate the diurnal cycle we want to group the data by `groupby` the hour and calculate the mean `.mean()`.

</details>

# 4. <a id='toc4_'></a>[Energy exchange with the surface](#toc0_)

An important factor which influences the state of the atmosphere, and hence the weather, is its interaction with the surface. This interaction can be represented as the exchange of energy in the form of fluxes. We consider three types of energy exchange: radiation, sensible heat flux and latent heat flux. The variable names corresponding to these fluxes are respectively `rnetds`, `hfss` and `hfls`.

- Load in the simulation data of these flux variables for the `initSFX`-simulation.
- What are the units of these fluxes?
- Select again the data from Uccle and from the period between 22 and 26 July 2019. Plot the ernergy fluxes in function of time. 
- We will now determine the sign of these fluxes:
    - The radiation flux `rnetds` is the net radiation flux: it is the difference between the incoming and outgoing radiation. The incoming radiation is primarily shortwave radiation: either direct (from the Sun) or diffuse (scattered solar radiation). The outgoing radiation is emmited by the surface itself; it is longwave radiation. The difference between these two contributions is the net radiation flux. When is this variable positive and negative? What does this mean: is a positive flux directed into or out of the surface?
    - The sensible heat flux `hfss` is the heat transfer caused by the temperature difference between the surface and the atmosphere. The heat is transfered from the medium with the highest temperature to the medium with the lowest temperature. Compare the near-surface air temperature (`tas`) with the surface temperature (`ts`). Use this difference between `ts` and `ts` to derive the sign of the sensible heat flux.
    - The latent heat flux is caused by the evaporation of soil moisture. This evaporation requires latent heat for the phase transition between the liquid and vapor phase. This heat is extracted from the soil, which causes it to cool. Consider the latent heat flux. In what direction is the postive latent heat flux defined? 
- Calculate the net energy flux into the surface by adding or subtracting (depending on how the direction is defined) the three flux contributions. Compare this with the surface temperature, stored in the variable `ts`.

<details><summary>Hints</summary> 

- The units of the variables can be found in the attributes.
- Selecting data in a point can be done with the `.sel()` function.
- To determine the meaning of the sign of the net radiation flux: look at which hour of the day the flux is positive.
- When the surface temperature `ts` is higher than the near-surface air temperature `tas`, then the sensible heat flux `hfss` will be pointed from the surface to the air.
- To calculate the net energy flux into the soil we should add all fluxes which are positive when going into the soil and subtract all fluxes which are negative going into the soil.

</details>

# 5. <a id='toc5_'></a>[Cold start vs. soil spin-up](#toc0_)

In this practical, we have discussed the interaction between the surface and the atmosphere through fluxes. The simulation we discussed previously, `initSFX`, has been run with an initial surface state that was taken from a long-term climate simulation. This allows the soil variables to start from a state that was in equilibrium with the atmospheric variables. As these soil variables (such as temperature and moisture) are slowly-evolving, particularly the deep soil variables, the time to reach this equilibrium in a simulation can be of the order of months.

On the other hand, we have also run a simulation with a default initial surface state (we call this a cold start). This simulation is the `baseline`-simulation (also used in the practical about the Vesder case). For this simulation, the soil moisture and temperature variables are not yet in an equilibrium state and will hence first evolve towards this state. This will impact the atmospheric variables close to the surface. The process where a model moves towards an equilibrium state is called the spin-up period.

Investigate this by following these steps:
- Load in the following variables for the `baseline`- and `initSFX`-simulations for the simulations starting on 1 July 2019 (720 hours):
    - `tas`: near-surface air temperature
    - `hfss`, `hfls`, `rnetds`: surface energy fluxes
    - `ts`: surface temperature
    - `mrso`: total soil moisture
    - `tsl2`: deep soil temperature
- Select the data at the locations of the MOCCA stations for the heatwave period (22-26 July 2019). Plot the temperature for both simulations and compare to the observations. Which simulation performs best?
- Plot the average over the simulation domain of all variables. How can you see the behaviour of moving to equilibrium from these plots? Can you explain the differences between these two simulations? Think about how these variables influence each other and explain how these differences are connected.

<details><summary>Hints</summary> 

- Loading in the data can be done with the `open_data` function by providing the correct `var_name_list` and `run_name_list` (with two simulations this time).
- Selecting data at the locations of the MOCCA stations can be done similarly to exercise 3.
- Plotting the temperature for both simulations can be done nicely with `col="run_name"`.
- The average over the domain can be calculated with `.mean()` and by providing the dimensions to average over.
- A trick to plot all variables from a Dataset is to first covnert the Dataset to a Dataarray with `.to_dataarray()` and then to plot with `col="variable"`. To compare both simulations you can then use `hue="run_name"`.

</details>

# 6. <a id='toc6_'></a>[Extra 1. Locations with a heatwave](#toc0_)

This extra exercise concerns the definition of a heat wave, as employed by the Belgian Royal Meteorological Institute. There, a heat wave period is defined as a period when the maximum temperatures in Uccle reach at least 25 degrees for at least 5 consecutive days, with at least 30 degrees being reached on at least three days. Write an algorithm to automatically detect heatwaves based on daily maximum temperatures. It is best to write it in two steps: 1) Select all periods which have at least five consecutive days above 25°C. 2) Check which of these periods contain at least three days with a temperature above 30°C. 

- Apply this algorithm on every point in the simulation dataset. What is the maximum number of heatwave days that you find over the domain?
- Apply this to the CLIMATE-GRID dataset. The daily maximum temperature file is named: `CLIMATE_GRID_tasmax_2019_07_daily_latlon_5km.nc`. How long are the heat waves in this dataset?

# 7. <a id='toc7_'></a>[Extra 2. Simulation without urban areas](#toc0_)

The model which we have used for all simulations consists of an atmospheric part, ALARO, and a surface module, SURFEX. The interaction between the atmosphere and the surface is an important part of weather and climate modelling. These interactions depend heavily on the type of surface. SURFEX tackles this variety by considering four classes of surface: sea, (inland) water, town and nature. Based on an external dataset (ECOCLIMAP) every grid box of the model is assigned a certain fraction of each of these four classes. For example, for a model with a resolution of 4-km, we could find a grid box in a small town with 30 % town fraction and 70 % nature fraction. Also, a grid box containing a city by the coast could have a town fraction of 25 %, a nature fraction of 20 % and a sea fraction of 55 %. Note that the fractions for each grid box necessarily always sum to 100 %. Every part of the grid box is calculated separately by a different sub-module, specific to the surface type. The output of these calculations (for example the near-surface air temperature) are then aggregated for the whole grid box, by taking a weighted average, with the land use fractions taken as weights. 

We will now investigate a simulation where we made some changes to the land use. For this simulation, we have taken every urban area and put it equal to rock. Normally, urban areas are calculated by the TEB (Town Energy Balance)-module, therefore we denote this simulation by `noTEB`. Explore this simulation by following the next steps:
- Load in the `tas` (near-surface air temperature) data for the simulations `initSFX` (with towns) and `initSFXnoTEB` (without towns) for the simulation starting on 1 July 2019 (720 hours). Select the period of the heat wave: 22 until 26 July 2019. 
- Load in the `land_use` data for both simulations. Plot all fractions for both simulations and verify the differences are as expected.
- Select only the data from points which have a town fraction higher than 20 % in the `initSFX`-simulations. Select these points for both simulations. In the `initSFXnoTEB`-simulation these points will have a 0 % town fraction.
- Calculate the average diurnal cycle of the temperature averaged over all points for both simulations. The average diurnal (or daily) cycle is calculated by taking the average of a variable for each hour of the day. The `groupby`-function from xarray will prove useful for this part.
- Compare the diurnal cycles for both simulations. What do you see? Plot the difference between both cycles. When is the difference largest?